# Data cleaning and operationalization

I'm going to introduce a dataset of vehicle crashes in Indiana, show how to import it into pandas, and give examples of a few ways of exploring this data.

**Download the data from [here](https://github.com/jdfoote/Intro-to-Programming-and-Data-Science/blob/master/resources/data/aries_crash_data_2018_filtered.csv?raw=true)**. I created this CSV file from [this full dataset](https://hub.mph.in.gov/dataset/aries-crash-data-2007-2017/resource/cc90589c-72d8-4d92-a5fe-73254b555c73) by filtering out many of the columns to make it small enough to easily fit in memory.

A description of the columns is [here](https://hub.mph.in.gov/dataset/aries-crash-data-2007-2017/resource/f61a5dcb-5ca3-485a-9ecf-cd3d8740dc9b?inner_span=True).

Then, I want you to form groups of 2-4 where you identify a hypothesis about this data, brainstorm a visualization or set of visualizations that would give you insight into your hypothesis, and then try to create that visualization in Python.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# This code downloads the CSV every time it's run.
# Instead, you may want to download the CSV to your computer, and load it from there
# You can use the same code, just change the path to where you download it
raw_df = pd.read_csv('https://github.com/jdfoote/Intro-to-Programming-and-Data-Science/blob/master/resources/data/aries_crash_data_2018_filtered.csv?raw=true')

## Understanding and Cleaning the data

I always start by looking at the data to make sure things look reasonable and to figure out what we are looking at.

In [ ]:
raw_df

It looks like Gender and age and drug test results are all NA for the first few rows. Let's check how often they were recorded.

In [ ]:
raw_df.isna().sum()

In [ ]:
# This is the total number of rows
len(raw_df)

So, it looks like the drug test column "RESULTDRUGIND" is going to be pretty useless. It's almost never recorded.

It looks like age and gender are only recorded about half the time. Let's look a littel closer into the data and see if we can figure out what's going on.

What does this data represent?

It's still not totally clear what this data represents. From the description, we can see that the Individual MR Record refers to each incident, so let's so how many of them there are:

In [ ]:
len(raw_df.INDIVIDUAL_MR_RECORD.unique())

Far fewer than the total rows. Let's sort by that, and see if we can figure out more.

In [ ]:
raw_df.sort_values("INDIVIDUAL_MR_RECORD")

Let's look at just one incident; it looks like the drivers, any injured passengers, and the owners are all listed (although no information is given about owners).

In [ ]:
raw_df.loc[raw_df.INDIVIDUAL_MR_RECORD == 903070479,]

Let's see if this is true more broadly. It seems like the "Owner Vehicle" rows just aren't going to be much use to us at all. Let's try to get a sense of how common they are, and if they really never contain information

In [ ]:
len(raw_df[raw_df.PERSONTYPEDESCR == 'Owner Vehicle'])

In [ ]:
raw_df[raw_df.PERSONTYPEDESCR == 'Owner Vehicle'].isna().sum()

Looks like for the data we have here, the "Owner Vehicle" entries aren't giving us any extra information, so let's just remove them.

In [ ]:
raw_df = raw_df[raw_df.PERSONTYPEDESCR != 'Owner Vehicle']

## Creating measures from raw data

It is very common that the raw data isn't in the form that you need it. You need to figure out how to transform it into something usable for the statistics and/or visualizations that you want to produce.

In this case, I'm going to take the goal of understanding collisions. Each row of the data represents a person, not a collision, so if we visualized or analyzed the raw data, we could be misled, as collisions with more people would receive more weight.

So, what we need to do is to build a dataset of accidents, with datetime, weather, number injured, and number killed.

My suspicion, based on what I've seen so far, is that all of these are the same across each of the rows for a given collision. It is worth testing that assumption, though.

In [ ]:
grouped = raw_df.sort_values(['COLLDTE', 'COLLISION_TIME']).groupby("INDIVIDUAL_MR_RECORD")

# For example, this is a simple test of the number of unique weather descriptions per group.
sum(grouped.WEATHERDESCR.nunique() > 1)

In [ ]:
# We see the same for number injured and number killed
print(sum(grouped.INJUREDNMB.nunique() > 1))
print(sum(grouped.DEADNMB.nunique() > 1))

So, we can move forward with the assumption that just getting the first value is good enough for summarizing the accident.

In [ ]:
# Get the first entry for each of these
coll_df = pd.DataFrame({"date" : grouped.COLLDTE.first(),
                        "time": grouped.COLLISION_TIME.first() + grouped.COLLISION_TIME_AM_PM.first(), 
                        "injured": grouped.INJUREDNMB.first(),
                        "deceased": grouped.DEADNMB.first(),
                        "weather": grouped.WEATHERDESCR.first(),
                        "accident_count": 1
                       })

coll_df

In [ ]:
coll_df.isna().sum()

In [ ]:
len(coll_df)

## Visualizing our data

We finally(!) have a dataset that we understand and that we can use in order to start to answer the questions we're interested in.

As before, we need to get the date and time into a datetime object in the index.

`pd.to_datetime` tries to convert a string into a datetime. I had to add a space in between the two columns to get it to work, but this does the trick.

In [ ]:
coll_df.index = pd.to_datetime(coll_df.date + ' ' + coll_df.time)

Let's look at the total number of accidents by month over 2018.

In [ ]:
coll_df.resample('M').accident_count.sum().plot()

And how about the relationship between weather and accidents?

In [ ]:
weather_summary = coll_df.groupby('weather').accident_count.sum()
weather_summary.plot.bar();

How could we correct for how often weather appears?

One simple way is to categorize each day by the weather that appears most often in accident reports. This obviously has some issues but it's the best we can probably do using just this data. What could be wise is to import another dataset which has historical weather data.

In [ ]:
accidents_per_day = coll_df.groupby(coll_df.index.date).agg(
    # Create a new column called weather which gets the modal weather for each day
    weather = ('weather', lambda x: x.mode()),
    # And accidents which is the number of accidents that day
    accidents = ('accident_count', sum))

In [ ]:
accidents_per_day.groupby('weather').accidents.mean().plot.bar();

## Exercises

1. How would you create the above plot in Seaborn instead?

2. Find a question that you'd like to answer with this data or the reddit data. Figure out how to filter/clean/group the data to produce the summary data that will help you to find that answer, and produce a visualization.